In [7]:
import json
from collections import Counter
from itertools import repeat
import pandas as pd
from tqdm import tqdm
import numpy as np

In [103]:
with open("Test/Test_frequency_bin.json", "r") as f:
    json_file = json.load(f)
open_word_tag = ["JJ", "JJR", "JJS", "RB", "RBR", "RBS", "NN", "NNS", "NNP", "NNPS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "FW"]
open_tag_dict = {key:json_file[key] for key in open_word_tag}

In [104]:
def filter_words(pos_dict, K):
    words_full = []
    for keys in pos_dict.keys():
        words_full.extend(pos_dict[keys])
    words_count = Counter(words_full)
    output = [word for item, n in words_count.items() if n < K for word in repeat(item, n)]
    for keys in tqdm(pos_dict.keys()):
        for word in pos_dict[keys]:
            if word in output:
                pos_dict[keys].remove(word)
    return pos_dict

filtered_words = filter_words(open_tag_dict, 10)

100%|██████████| 17/17 [00:13<00:00,  1.30it/s]


In [105]:
def pos_tag_filter(pos_dict, N):
    full_words = []
    for key in pos_dict.keys():
        full_words.extend(pos_dict[key])
    full_words = list(set(full_words))
    tmp = {key:list(set(pos_dict[key])) for key in pos_dict.keys()}
    ambiguous_words = []
    for word in full_words:
        cnt = 0
        for key in tmp.keys():
            if word in tmp[key]:
                cnt += 1
            if cnt >= N:
                ambiguous_words.append(word)
                break

    return ambiguous_words

In [106]:
ambiguous_words = pos_tag_filter(filtered_words, 2)
ambiguous_words_pos = {}
for key in open_word_tag:
    ambiguous_words_pos[key] = [word for word in filtered_words[key] if word in ambiguous_words]

In [108]:
n = len(ambiguous_words)
k = len(ambiguous_words_pos.keys())
df = pd.DataFrame(np.zeros((n, k)), columns=open_word_tag, index=ambiguous_words)

In [109]:
number_of_total_words = sum([len(ambiguous_words_pos[key]) for key in ambiguous_words_pos.keys()])
print(number_of_total_words)

29623


In [111]:
for tag in open_word_tag:
    words_count = Counter(ambiguous_words_pos[tag])
    vals = np.zeros(n)
    for indx, word in enumerate(ambiguous_words):
        vals[indx] = words_count.get(word, 0.0)
    df[tag] = vals

    

In [112]:
df.to_csv("Test/Test_word_amb_tag.csv", index=True)

In [132]:
word_tag_matrix = df.values
tag_prob = np.sum(word_tag_matrix, axis=0)
word_norm = (word_tag_matrix.T/word_tag_matrix.sum(axis=1)).T
word_tag_norm = word_norm / tag_prob
word_score = -np.sum(word_norm * np.log2(word_tag_norm, out=np.zeros_like(word_tag_norm), where=(word_tag_norm!=0)), axis=1)

res = pd.DataFrame(np.zeros((n, 2)), columns=["Word", "Score"])
res["Word"] = ambiguous_words
res["Score"] = word_score
res = res.sort_values(by=["Score"], ascending=False)
res.to_csv('Test/Test_word_amb_score.csv', index=False)



In [116]:
print(sum((np.sum(word_tag_matrix, axis=1)/number_of_total_words)))

1.0


In [139]:
word_to_test = "friar"
score = 0
m = 0
n = 0
for tag in open_word_tag:
    val = ambiguous_words_pos[tag].count(word_to_test)
    score += val
    if val != 0:
        print(val)
        print(tag)
        n += 1
    if val > m:
        m = val
print(score, n, m)

0 0 0


In [11]:
word_counts = Counter(json_file["NNPS"])
df = pd.DataFrame.from_dict(word_counts, orient='index')
print(df.sort_values(by=[0], ascending=False))


              0
jews        313
americans   202
christians  165
islands     163
ages        138
...         ...
celtics       1
heroes        1
sensations    1
maldives      1
problems      1

[1231 rows x 1 columns]
